# Read SAS Files
The following data can be found here:
https://wwwn.cdc.gov/nchs/nhanes/

In [1]:
# Read SAS file
import pandas as pd

In [2]:
# NHANES 2001-2002 vaccination Immunization
nhanes0304_Vaccination = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/IMQ_C.XPT", format = "xport")

In [3]:
# NHANES 2001-2002 core antibody surface antigen
nhanes0304_BG = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/L02_C.XPT", format = "xport" )

In [4]:
# NHANES 2001-2002 surface antibody
nhanes0304_Antibody = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/L02HBS_C.XPT", format = "xport")

In [5]:
nhanes0304_Vaccination = nhanes0304_Vaccination[['SEQN','IMQ020']]

In [6]:
nhanes0304_BG = nhanes0304_BG[['SEQN','LBXHBC','LBDHBG']]

In [7]:
nhanes0304_Antibody

,SEQN,LBXHBS
0,21005.0,2.0
1,21006.0,2.0
2,21007.0,1.0
3,21008.0,1.0
4,21009.0,2.0
...,...,...
8842,31121.0,2.0
8843,31122.0,NaN
8844,31123.0,1.0
8845,31124.0,2.0


In [8]:
nhanes0304_No_vaccination = nhanes0304_Vaccination[(nhanes0304_Vaccination['IMQ020']==3.0) | (nhanes0304_Vaccination['IMQ020']==7.0) | (nhanes0304_Vaccination['IMQ020']==9.0)]

In [9]:
nhanes0304_No_vaccination

,SEQN,IMQ020
0,21005.0,3.0
4,21009.0,3.0
5,21010.0,3.0
6,21011.0,9.0
7,21012.0,3.0
...,...,...
10114,31119.0,3.0
10116,31121.0,3.0
10118,31123.0,3.0
10119,31124.0,3.0


In [10]:
nhanes0304_club_1 = pd.merge(nhanes0304_No_vaccination, nhanes0304_Antibody, on="SEQN", how = "left")

In [11]:
nhanes0304_club_2 = pd.merge(nhanes0304_club_1, nhanes0304_BG, on="SEQN", how = "left")

In [12]:
import pandas as pd
import pandas.io.sql as sqlio

In [13]:
pip install pandasql

Note: you may need to restart the kernel to use updated packages.


In [14]:
from pandasql import sqldf

In [15]:
query = '''
SELECT SEQN, (CASE WHEN LBXHBC = 1.0 THEN 'Positive' WHEN LBXHBC = 2.0 THEN 'Negative' WHEN LBXHBC IS NULL THEN 'Missing' END) AS Core_Antibody,
(CASE WHEN LBDHBG = 1.0 THEN 'Positive' WHEN LBDHBG = 2.0 THEN 'Negative' WHEN LBDHBG IS NULL THEN 'Missing' END) AS Surface_Antigen,
(CASE WHEN LBXHBS = 1.0 THEN 'Positive' WHEN LBXHBS = 2.0 THEN 'Negative' WHEN LBXHBS IS NULL THEN 'Missing' END) AS Surface_Antibody,
(CASE WHEN IMQ020 = 3.0 THEN 'No Doses' WHEN IMQ020 = 7.0 THEN 'Refused' WHEN IMQ020 = 9.0 THEN "Don't know" END) AS Vaccination
FROM nhanes0304_club_2
'''

In [16]:
nhanes0304_final = sqldf(query)

In [17]:
nhanes0304_final[nhanes0304_final['Surface_Antigen'] == 'Positive']

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination
462,21995.0,Positive,Positive,Negative,No Doses
478,22032.0,Positive,Positive,Negative,Don't know
780,22681.0,Positive,Positive,Negative,No Doses
1220,23582.0,Positive,Positive,Negative,No Doses
1322,23789.0,Positive,Positive,Negative,No Doses
1419,23992.0,Positive,Positive,Negative,No Doses
1683,24494.0,Positive,Positive,Negative,No Doses
1704,24529.0,Positive,Positive,Negative,No Doses
2451,26114.0,Positive,Positive,Positive,No Doses
2785,26809.0,Positive,Positive,Negative,No Doses


In [18]:
query2 = '''
SELECT SEQN, Core_Antibody, Surface_Antigen, Surface_Antibody, Vaccination, 
(CASE WHEN Surface_Antigen = 'Positive' THEN 'HBV Infection' WHEN (Surface_Antibody = 'Positive') THEN 'Vaccine Immunity' 
WHEN (Surface_Antigen = 'Negative' and Core_Antibody ='Negative' and Surface_Antibody ='Negative') THEN 'No Infection'
WHEN (Core_Antibody = 'Positive' and Surface_Antibody = 'Negative') THEN 'HBV Exposure' END) AS resp_2 
FROM nhanes0304_final
'''

In [19]:
nhanes0304_final_resp = sqldf(query2)

In [20]:
nhanes0304_final_resp[nhanes0304_final_resp['resp_2'].isna()]

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2
3,21011.0,Missing,Missing,Missing,Don't know,None
7,21018.0,Missing,Missing,Missing,No Doses,None
10,21021.0,Missing,Missing,Missing,No Doses,None
12,21028.0,Missing,Missing,Missing,No Doses,None
13,21030.0,Missing,Missing,Missing,No Doses,None
...,...,...,...,...,...,...
4894,31084.0,Missing,Missing,Missing,No Doses,None
4902,31100.0,Missing,Missing,Missing,No Doses,None
4904,31103.0,Missing,Missing,Missing,No Doses,None
4905,31104.0,Missing,Missing,Missing,Don't know,None


In [21]:
# NHANES 2017-2018 Hepatitis ever told
#nhanes9920_evertold = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/1999-2000/HEQ_H.XPT", format = "xport")

In [22]:
#nhanes1314_evertold = nhanes1314_evertold[['SEQN','HEQ010']]

In [23]:
# left outer join prior data with evertold
#nhanes1314_final_resp_comb = pd.merge(nhanes1314_final_resp, nhanes1314_evertold, on="SEQN", how = "left")

In [24]:
#nhanes1314_final_resp_comb

In [25]:
#query3 = '''
#SELECT SEQN, Core_Antibody, Surface_Antigen, Surface_Antibody, Vaccination, resp, HEQ010,
#(CASE WHEN resp IS NULL and HEQ010 = 1.0 THEN 'HBV Infection' WHEN resp IS NULL and HEQ010 = 2.0 THEN 'No Infection' ELSE resp END) AS resp_2
#FROM nhanes1314_final_resp_comb
#'''

In [26]:
#nhanes9920_final_resp_data = sqldf(query3)

In [27]:
nhanes0304_HBV = nhanes0304_final_resp[(nhanes0304_final_resp['resp_2']=='HBV Infection') | (nhanes0304_final_resp['resp_2']=='No Infection')]

In [28]:
nhanes0304_HBV

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2
0,21005.0,Negative,Negative,Negative,No Doses,No Infection
1,21009.0,Negative,Negative,Negative,No Doses,No Infection
2,21010.0,Negative,Negative,Negative,No Doses,No Infection
4,21012.0,Negative,Negative,Negative,No Doses,No Infection
5,21015.0,Negative,Negative,Negative,No Doses,No Infection
...,...,...,...,...,...,...
4912,31116.0,Negative,Negative,Negative,No Doses,No Infection
4913,31119.0,Negative,Negative,Negative,No Doses,No Infection
4914,31121.0,Negative,Negative,Negative,No Doses,No Infection
4916,31124.0,Negative,Negative,Negative,No Doses,No Infection


In [29]:
# NHANES 2017-2018 Demographic Variable
nhanes0304_Demographic = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/DEMO_C.XPT", format = "xport")

In [30]:
#changed columns 'RIDRETH1','DMQMILIT','DMDBORN', deleted 'DMQADFC' 
Demographic_Variables = ['SEQN','SDDSRVYR','RIAGENDR','RIDAGEYR','RIDRETH1','DMQMILIT','DMDBORN','DMDCITZN',
                        'DMDYRSUS','DMDEDUC2','DMDMARTL','WTINT2YR','WTMEC2YR','SDMVPSU','SDMVSTRA','INDFMPIR']

In [31]:
nhanes0304_Demographic = nhanes0304_Demographic[Demographic_Variables]

In [32]:
nhanes0304_HBV_Demo = pd.merge(nhanes0304_HBV, nhanes0304_Demographic, on="SEQN", how = "left")

In [33]:
nhanes0304_HBV_Demo

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2,SDDSRVYR,RIAGENDR,RIDAGEYR,RIDRETH1,...,DMDBORN,DMDCITZN,DMDYRSUS,DMDEDUC2,DMDMARTL,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDFMPIR
0,21005.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,19.0,4.0,...,1.0,1.0,NaN,NaN,5.0,5512.320949,5824.782465,2.0,39.0,2.44
1,21009.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,55.0,3.0,...,1.0,1.0,NaN,3.0,1.0,97593.678977,97731.727244,2.0,31.0,3.79
2,21010.0,Negative,Negative,Negative,No Doses,No Infection,3.0,2.0,52.0,3.0,...,1.0,1.0,NaN,4.0,6.0,39599.362689,43286.576474,1.0,29.0,1.24
3,21012.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,63.0,4.0,...,1.0,1.0,NaN,3.0,6.0,12629.440480,12947.338142,2.0,33.0,0.89
4,21015.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,83.0,3.0,...,1.0,1.0,NaN,4.0,1.0,17625.875727,19994.159857,2.0,33.0,1.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441,31116.0,Negative,Negative,Negative,No Doses,No Infection,3.0,2.0,54.0,3.0,...,1.0,1.0,NaN,4.0,1.0,72315.328639,78940.852203,1.0,35.0,5.00
3442,31119.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,23.0,1.0,...,2.0,2.0,2.0,2.0,5.0,30032.816129,29947.658360,1.0,39.0,3.93
3443,31121.0,Negative,Negative,Negative,No Doses,No Infection,3.0,2.0,68.0,1.0,...,1.0,1.0,NaN,2.0,1.0,3339.727014,3385.682094,2.0,40.0,2.08
3444,31124.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,44.0,1.0,...,2.0,2.0,4.0,2.0,1.0,17304.950958,17496.954286,1.0,39.0,3.93


In [34]:
nhanes0304_Insurance = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/HIQ_C.XPT", format = "xport")

In [35]:
Insurance_Variables = ['SEQN','HID010','HID030A','HID030B','HID030C','HID030D','HID030E','HID040','HIQ210','HIQ220']

In [36]:
nhanes0304_Insurance = nhanes0304_Insurance[Insurance_Variables]

In [37]:
nhanes0304_HBV_Demo_Insurance = pd.merge(nhanes0304_HBV_Demo, nhanes0304_Insurance, on="SEQN", how = "left")

In [38]:
query4 = '''
SELECT *, 
(CASE WHEN HID010 = 2.0 THEN 1 ELSE 0 END) AS No_Insurance,
(CASE WHEN HID010 = 1.0 and HID030A = 1.0 THEN 1 ELSE 0 END) AS Private_Insurance,
(CASE WHEN HID010 = 1.0 and (HID030A IS NOT NULL OR HID030B IS NOT NULL OR HID030C IS NOT NULL OR HID030D IS NOT NULL OR 
HID030E IS NOT NULL) THEN 1 ELSE 0 END) Governement_Insurance
FROM nhanes0304_HBV_Demo_Insurance
'''

In [39]:
nhanes0304_HBV_Demo_Insurance = sqldf(query4)

In [40]:
nhanes0304_HBV_Demo_Insurance = nhanes0304_HBV_Demo_Insurance.drop(columns=['HID010','HID030A','HID030B','HID030C','HID030D','HID030E','HID040','HIQ210','HIQ220'])

In [41]:
nhanes0304_HBV_Demo_Insurance

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2,SDDSRVYR,RIAGENDR,RIDAGEYR,RIDRETH1,...,DMDEDUC2,DMDMARTL,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDFMPIR,No_Insurance,Private_Insurance,Governement_Insurance
0,21005.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,19.0,4.0,...,NaN,5.0,5512.320949,5824.782465,2.0,39.0,2.44,0,1,1
1,21009.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,55.0,3.0,...,3.0,1.0,97593.678977,97731.727244,2.0,31.0,3.79,0,1,1
2,21010.0,Negative,Negative,Negative,No Doses,No Infection,3.0,2.0,52.0,3.0,...,4.0,6.0,39599.362689,43286.576474,1.0,29.0,1.24,1,0,0
3,21012.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,63.0,4.0,...,3.0,6.0,12629.440480,12947.338142,2.0,33.0,0.89,1,0,0
4,21015.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,83.0,3.0,...,4.0,1.0,17625.875727,19994.159857,2.0,33.0,1.20,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441,31116.0,Negative,Negative,Negative,No Doses,No Infection,3.0,2.0,54.0,3.0,...,4.0,1.0,72315.328639,78940.852203,1.0,35.0,5.00,0,1,1
3442,31119.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,23.0,1.0,...,2.0,5.0,30032.816129,29947.658360,1.0,39.0,3.93,0,1,1
3443,31121.0,Negative,Negative,Negative,No Doses,No Infection,3.0,2.0,68.0,1.0,...,2.0,1.0,3339.727014,3385.682094,2.0,40.0,2.08,0,1,1
3444,31124.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,44.0,1.0,...,2.0,1.0,17304.950958,17496.954286,1.0,39.0,3.93,1,0,0


In [42]:
#Alcohol use
nhanes0304_Alcohol = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/ALQ_C.XPT", format = "xport")

In [43]:
nhanes0304_Alcohol = nhanes0304_Alcohol[['SEQN','ALQ130']]

In [44]:
nhanes0304 = pd.merge(nhanes0304_HBV_Demo_Insurance, nhanes0304_Alcohol, on="SEQN", how = "left")

In [45]:
#Healthcare Visits
nhanes0304_Healthcare = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/HUQ_C.XPT", format = "xport")

In [46]:
nhanes0304_Healthcare = nhanes0304_Healthcare [['SEQN','HUQ050']]

In [47]:
nhanes0304 = pd.merge(nhanes0304, nhanes0304_Healthcare, on="SEQN", how = "left")

In [48]:
#Dialysis
nhanes0304_Dialysis = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/KIQ_U_C.XPT", format = "xport")

In [49]:
nhanes0304_Dialysis = nhanes0304_Dialysis[['SEQN','KIQ025']]

In [50]:
nhanes0304 = pd.merge(nhanes0304, nhanes0304_Dialysis, on="SEQN", how = "left")

In [51]:
#Liver Condition
nhanes0304_Liver_Condition = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")

C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Liver_Condition = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Liver_Condition = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py

C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Liver_Condition = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Liver_Condition = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py

C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Liver_Condition = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Liver_Condition = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\1061492775.py

In [52]:
nhanes0304_Liver_Condition = nhanes0304_Liver_Condition[['SEQN','MCQ160L']]

In [53]:
nhanes0304 = pd.merge(nhanes0304, nhanes0304_Liver_Condition, on="SEQN", how = "left")

In [54]:
#Blood Transfusion
nhanes0304_Blood_Transfusion = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")

C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Blood_Transfusion = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Blood_Transfusion = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.p

C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Blood_Transfusion = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Blood_Transfusion = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.p

C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Blood_Transfusion = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nhanes0304_Blood_Transfusion = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/MCQ_C.XPT", format = "xport")
C:\Users\Kirtikanth\AppData\Local\Temp\ipykernel_27048\697513097.p

In [55]:
nhanes0304_Blood_Transfusion = nhanes0304_Blood_Transfusion[['SEQN','MCQ092']]

In [56]:
nhanes0304 = pd.merge(nhanes0304, nhanes0304_Blood_Transfusion, on="SEQN", how = "left")

In [57]:
#Healthcare Visits time
nhanes0304_Healthcare_time = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/HUQ_C.XPT", format = "xport")

In [58]:
nhanes0304_Healthcare_time = nhanes0304_Healthcare_time[['SEQN','HUQ060']]

In [59]:
nhanes0304 = pd.merge(nhanes0304, nhanes0304_Healthcare_time, on="SEQN", how = "left")

In [60]:
#Hepatitis C
nhanes0304_Hepatitis_C = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/HCQ_C.XPT", format = "xport")

In [61]:
nhanes0304_Hepatitis_C = nhanes0304_Hepatitis_C[['SEQN']]

In [62]:
import numpy as np
nhanes0304_Hepatitis_C['Hepatitis_C'] = "Yes"

In [63]:
nhanes0304 = pd.merge(nhanes0304, nhanes0304_Hepatitis_C, on="SEQN", how = "left")

In [64]:
Biochemistry_Variables = ['SEQN','LBXSATSI','LBDSALSI','LBXSASSI']

In [65]:
#Biochemistry_Variables
nhanes0304_Biochemistry_Variables = pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/L40_C.XPT", format = "xport")

In [66]:
nhanes0304_Biochemistry_Variables = nhanes0304_Biochemistry_Variables[Biochemistry_Variables]

In [67]:
nhanes0304 = pd.merge(nhanes0304, nhanes0304_Biochemistry_Variables, on="SEQN", how = "left")

In [68]:
#Platelet count
nhanes0304_Platelet_count= pd.read_sas("https://wwwn.cdc.gov/Nchs/Nhanes/2003-2004/L25_C.XPT", format = "xport")

In [69]:
nhanes0304_Platelet_count = nhanes0304_Platelet_count[['SEQN','LBXPLTSI']]

In [70]:
nhanes0304 = pd.merge(nhanes0304, nhanes0304_Platelet_count, on="SEQN", how = "left")

In [71]:
nhanes0304 = nhanes0304[nhanes0304['RIDAGEYR']>=18]

In [72]:
pd.set_option('display.max_columns', None)
nhanes0304.head()

,SEQN,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2,SDDSRVYR,RIAGENDR,RIDAGEYR,RIDRETH1,DMQMILIT,DMDBORN,DMDCITZN,DMDYRSUS,DMDEDUC2,DMDMARTL,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDFMPIR,No_Insurance,Private_Insurance,Governement_Insurance,ALQ130,HUQ050,KIQ025,MCQ160L,MCQ092,HUQ060,Hepatitis_C,LBXSATSI,LBDSALSI,LBXSASSI,LBXPLTSI
0,21005.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,19.0,4.0,2.0,1.0,1.0,NaN,NaN,5.0,5512.320949,5824.782465,2.0,39.0,2.44,0,1,1,NaN,1.000000e+00,NaN,NaN,2.0,NaN,NaN,15.0,37.0,17.0,314.0
1,21009.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,55.0,3.0,2.0,1.0,1.0,NaN,3.0,1.0,97593.678977,97731.727244,2.0,31.0,3.79,0,1,1,NaN,5.397605e-79,NaN,2.0,2.0,3.0,NaN,22.0,43.0,20.0,160.0
2,21010.0,Negative,Negative,Negative,No Doses,No Infection,3.0,2.0,52.0,3.0,2.0,1.0,1.0,NaN,4.0,6.0,39599.362689,43286.576474,1.0,29.0,1.24,1,0,0,4.0,1.000000e+00,NaN,2.0,2.0,NaN,NaN,34.0,42.0,28.0,364.0
3,21012.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,63.0,4.0,2.0,1.0,1.0,NaN,3.0,6.0,12629.440480,12947.338142,2.0,33.0,0.89,1,0,0,NaN,1.000000e+00,NaN,2.0,2.0,NaN,NaN,26.0,41.0,24.0,187.0
4,21015.0,Negative,Negative,Negative,No Doses,No Infection,3.0,1.0,83.0,3.0,1.0,1.0,1.0,NaN,4.0,1.0,17625.875727,19994.159857,2.0,33.0,1.20,0,0,1,NaN,3.000000e+00,NaN,2.0,1.0,NaN,NaN,32.0,41.0,37.0,159.0


In [73]:
nhanes0304.shape

(3259, 35)

In [74]:
nhanes0304 = nhanes0304.rename(columns = {'SEQN':'Respondent_sequence_number',
                            'SDDSRVYR':'Data_release_cycle',
                            'RIAGENDR':'Gender',
                            'RIDAGEYR':'Age',
                            'RIDRETH1':'Race_Ethnicity',
                            'DMQMILIT':'Veteran_Status',
                            'DMDBORN':'Country_of_birth',
                            'DMDCITZN':'Citizenship_status',
                            'DMDYRSUS':'Length_of_time_in_US',
                            'DMDEDUC2':'Education_level',
                            'DMDMARTL':'Marital_status',
                            'WTINT2YR':'Two_year_interview_weight',
                            'WTMEC2YR':'Two_year_MEC_weight',
                            'SDMVPSU':'Masked_variance_PSU',
                            'SDMVSTRA':'Masked_variance_stratum',
                            'INDFMPIR':'Ratio_income_poverty',
                            'ALQ130':'Alcohol_1_year',
                            'HUQ050':'Health_care_1_year',
                            'KIQ025':'Dialysis_1_year',
                            'MCQ160L':'liver_condition',
                            'MCQ092':'Blood_Transfusion',
                            'HUQ060':'Last_Healthcare_Visit',
                            'Hepatitis_C':'Hepatitis_C',
                            'LBXSATSI':'ALT',
                            'LBDSALSI':'Albumin',
                            'LBXSASSI':'AST',
                            'LBXPLTSI':'Platelet_Count'})

In [76]:
nhanes0304[nhanes0304['resp_2']=='HBV Infection'].shape

(20, 35)

In [77]:
column_mapping = {
    'Gender':{1.0:"male", 2.0:"Female", np.nan:"missing"},
    'Race_Ethnicity':{1.0:"Mexican American",2.0:"Other Hispanic",3.0:"Non-Hispanic White",4.0:"Non-Hispanic Black",5.0:"Other Race - Including Multi-Racial"},
    'Veteran_Status':{1.0:"Yes",2.0:"No",7.0:"Refused",9.0:"Don't know", np.nan:"missing"},
    'Country_of_birth':{1.0:"US", 2.0:"Mexico", 3.0:"Elsewhere",7.0:"Refused",9.0:"Don't know",np.nan:"missing"},
    'Citizenship_status':{1.0:"Citizen",2.0:"Not a citizen",7.0:"Refused",9.0:"Don't know",np.nan:"Missing"},
    'Length_of_time_in_US':{1.0:"Less than 1 year",2.0:"1-5",3.0:"5-10",4.0:"10-15",5.0:"15-20",6.0:"20-30",7.0:"30-40",8.0:"40-50",9.0:"50 or more",77.0:"Refused",88.0:"Could not determine",99.0:"Don't know",np.nan:"Missing"},
    'Education_level':{1.0:"Less Than 9th Grade",2.0:"9-12th Grade",3.0:"High School Grad/GED or Equivalent",4.0:"Some College or AA degree",5.0:"College Graduate or above",7.0:"Refused",9.0:"Don't Know",np.nan:"Missing"},
    'Marital_status':{1.0:"Married",2.0:"Widowed",3.0:"Divorced",4.0:"Separated",5.0:"Never married",6.0:"Living with partner",77.0:"Refused",99.0:"Don't know",np.nan:"Missing"},
    'Health_care_1_year':{5.397605e-79:"None",1.0:"1",2.0:"2 to 3",3.0:"4 to 9",4.0:"10 to 12",5.0:"13 or more",77.0:"Refused",99.0:"Don't know",np.nan:"Missing"},
    'Dialysis_1_year':{1.0:"Yes",2.0:"No",7.0:"Refused",9.0:"Don't know",np.nan:"Missing"},
    'liver_condition':{1.0:"Yes",2.0:"No",7.0:"Refused",9.0:"Don't know",np.nan:"Missing"},
    'Blood_Transfusion':{1.0:"Yes",2.0:"No",7.0:"Refused",9.0:"Don't know",np.nan:"Missing"},
    'Last_Healthcare_Visit':{1.0:"6 months or less",2.0:"More than 6 months, but not more than 1 year ago",3.0:"More than 1 year, but not more than 3 years ago",4.0:"More than 3 years",5.0:"Never",7.0:"Refused",9.0:"Don't know",np.nan:"Missing"},
    'Hepatitis_C':{'Yes':"Yes",np.nan:"Missing"}
}

nhanes0304.replace(column_mapping,inplace=True)

In [79]:
nhanes0304[nhanes0304['resp_2']=='HBV Infection']

,Respondent_sequence_number,Core_Antibody,Surface_Antigen,Surface_Antibody,Vaccination,resp_2,Data_release_cycle,Gender,Age,Race_Ethnicity,Veteran_Status,Country_of_birth,Citizenship_status,Length_of_time_in_US,Education_level,Marital_status,Two_year_interview_weight,Two_year_MEC_weight,Masked_variance_PSU,Masked_variance_stratum,Ratio_income_poverty,No_Insurance,Private_Insurance,Governement_Insurance,Alcohol_1_year,Health_care_1_year,Dialysis_1_year,liver_condition,Blood_Transfusion,Last_Healthcare_Visit,Hepatitis_C,ALT,Albumin,AST,Platelet_Count
327,21995.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,male,32.0,Non-Hispanic Black,No,Elsewhere,Not a citizen,1-5,Some College or AA degree,Separated,25175.844535,26177.729098,1.0,30.0,2.78,0,1,1,4.0,2 to 3,Missing,No,No,Missing,Missing,24.0,46.0,30.0,235.0
552,22681.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,male,22.0,Other Race - Including Multi-Racial,No,Elsewhere,Not a citizen,15-20,High School Grad/GED or Equivalent,Never married,76523.024194,81271.592246,2.0,37.0,3.83,0,1,1,NaN,1,Missing,No,No,Missing,Missing,24.0,41.0,31.0,216.0
858,23582.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,male,30.0,Other Race - Including Multi-Racial,No,Elsewhere,Not a citizen,10-15,9-12th Grade,Never married,39063.541787,43159.868623,2.0,41.0,NaN,0,0,1,NaN,0.0,Missing,No,No,More than 3 years,Missing,31.0,41.0,21.0,290.0
927,23789.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,male,53.0,Non-Hispanic Black,No,US,Citizen,Missing,9-12th Grade,Divorced,24885.244824,29080.281798,2.0,35.0,0.70,0,0,1,2.0,Don't know,Missing,No,Yes,Missing,Missing,16.0,38.0,22.0,197.0
998,23992.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,male,48.0,Non-Hispanic Black,No,Elsewhere,Not a citizen,10-15,High School Grad/GED or Equivalent,Married,25537.235547,26723.146264,1.0,36.0,NaN,1,0,0,4.0,13 or more,No,No,No,Missing,Missing,21.0,40.0,18.0,317.0
1171,24494.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,male,46.0,Non-Hispanic Black,No,US,Citizen,Missing,High School Grad/GED or Equivalent,Married,23269.944900,24350.566058,1.0,30.0,0.28,1,0,0,2.0,10 to 12,Missing,No,No,Missing,Missing,17.0,45.0,23.0,145.0
1187,24529.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,Female,50.0,Non-Hispanic Black,No,US,Citizen,Missing,High School Grad/GED or Equivalent,Widowed,30882.439273,31834.094118,1.0,32.0,1.11,0,0,1,NaN,13 or more,Missing,No,No,Missing,Missing,20.0,44.0,27.0,292.0
1704,26114.0,Positive,Positive,Positive,No Doses,HBV Infection,3.0,Female,54.0,Other Race - Including Multi-Racial,No,Elsewhere,Not a citizen,Less than 1 year,Less Than 9th Grade,Married,40838.011885,43523.401839,2.0,37.0,3.65,0,0,1,NaN,10 to 12,Missing,No,No,Missing,Missing,19.0,43.0,20.0,295.0
1923,26809.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,Female,46.0,Non-Hispanic Black,No,US,Citizen,Missing,Some College or AA degree,Never married,27525.778970,28373.996972,2.0,33.0,5.00,0,1,1,2.0,1,Missing,No,No,Missing,Missing,17.0,36.0,15.0,407.0
2030,27115.0,Positive,Positive,Negative,No Doses,HBV Infection,3.0,male,22.0,Other Race - Including Multi-Racial,No,Elsewhere,Not a citizen,1-5,Some College or AA degree,Never married,31277.823055,33272.966370,2.0,43.0,2.15,0,1,1,2.0,10 to 12,Missing,Yes,Yes,Missing,Missing,87.0,47.0,48.0,256.0
